# Como realizar el analisis de datos en Jupyter 

[PyCon Ar en PyVideo.org](https://pyvideo.org/events/pycon-ar-2011.html)

- Explorar los datos

In [19]:
import pathlib

carpeta_datos = pathlib.Path('./datos')

In [25]:
[ elemento for elemento in carpeta_datos.glob('*/*')]

[WindowsPath('datos/pycon-ar-2011/category.json'),
 WindowsPath('datos/pycon-ar-2011/videos'),
 WindowsPath('datos/pycon-ar-2012/category.json'),
 WindowsPath('datos/pycon-ar-2012/videos'),
 WindowsPath('datos/pycon-ar-2013/category.json'),
 WindowsPath('datos/pycon-ar-2013/videos'),
 WindowsPath('datos/pycon-ar-2014/category.json'),
 WindowsPath('datos/pycon-ar-2014/videos'),
 WindowsPath('datos/pycon-ar-2015/category.json'),
 WindowsPath('datos/pycon-ar-2015/videos'),
 WindowsPath('datos/pycon-ar-2016/category.json'),
 WindowsPath('datos/pycon-ar-2016/videos'),
 WindowsPath('datos/pycon-ar-2018/category.json'),
 WindowsPath('datos/pycon-ar-2018/videos')]

- Importar los datos con pandas

- Realizar graficos